In [1]:
import time

import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

from Conceptualizer import Conceptualizer
from LDA import LDA


def evaluate(word1, word2, sentence1, sentence2):
    concept_distribution1 = conceptualizer.conceptualize(sentence1, word1, eval=True)
    concept_distribution2 = conceptualizer.conceptualize(sentence2, word2, eval=True)
    if concept_distribution1 is None or concept_distribution2 is None:
        return None

    vector1 = np.zeros(np.max([len(concept_distribution1), len(concept_distribution2)]))
    vector2 = np.zeros(np.max([len(concept_distribution1), len(concept_distribution2)]))
    if len(concept_distribution1) == np.max([len(concept_distribution1), len(concept_distribution2)]):
        counter = 0
        for concept1 in concept_distribution1:
            vector1[counter] = concept1[1]
            for concept2 in concept_distribution2:
                if concept1[0] == concept2[0]:
                    vector2[counter] = concept2[1]
                    break
            counter += 1
    elif len(concept_distribution2) == np.max([len(concept_distribution1), len(concept_distribution2)]):
        counter = 0
        for concept2 in concept_distribution2:
            vector2[counter] = concept2[1]
            for concept1 in concept_distribution1:
                if concept2[0] == concept1[0]:
                    vector1[counter] = concept1[1]
                    break
            counter += 1
    else:
        print("jl")
        print(vector1)
        print(vector2)
        print(concept_distribution1)
        print(concept_distribution2)
        print(word1, sentence1)
        print(word2, sentence2)
        exit()
    try:
        estimated_similarity = cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))
    except ValueError as e:
        print(vector1)
        print(vector2)
        print(concept_distribution1)
        print(concept_distribution2)
        print(word1, sentence1)
        print(word2, sentence2)
        print(vector1.reshape(1, -1))
        print(vector2.reshape(1, -1))
        raise e

    return estimated_similarity


def get_eval_set():
    eval_set = []
    with(open('../ratings.txt', 'r')) as file:
        for line in file.readlines():
            splitted_line = line.split('\t')
            id = splitted_line[0]
            word1 = splitted_line[1]
            pos_word1 = splitted_line[2]
            word2 = splitted_line[3]
            pos_word2 = splitted_line[4]
            word1_in_context = splitted_line[5]
            word2_in_context = splitted_line[6]
            average_human_rating = splitted_line[7]
            ten_individual_human_ratings = [rating.replace('\n', '') for rating in splitted_line[7:len(splitted_line)]]
            word1_in_context = word1_in_context.replace('<b> ', '').replace(' </b>', '')
            word2_in_context = word2_in_context.replace('<b> ', '').replace(' </b>', '')
            eval_set.append((id, word1, word2, word1_in_context, word2_in_context, average_human_rating))
    return eval_set

C:\Users\Nicolas\Anaconda3\envs\kbqa\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
start_time = time.time()

MODEL_BASE_DIR = 'F:/Not_Uploaded/conceptualization_eval/models/'
MODEL_FILE_EXTENSION = '.gensim'
model_names = [
    # 'ldamodel_50',
    'ldamodel_simple_mallet_20_10_keep_300000_gensimstop_topics200'
]
model_stat = []
eval_set = get_eval_set()
for model_name in model_names:
    print("\nTest", model_name, ':')
    lda = LDA()
    lda.load(MODEL_BASE_DIR + model_name + MODEL_FILE_EXTENSION)
    conceptualizer = Conceptualizer(lda)

    none_counter = 0
    estimated_similarities = []
    real_similarities = []

    for element in eval_set:
        estimated_similarity = evaluate(element[1], element[2], element[3], element[4])
        if estimated_similarity is None:
            none_counter += 1
            continue
        estimated_similarities.append(float(estimated_similarity[0][0]))
        real_similarities.append(float(element[5]))
        if (int(element[0]) - 1) % 100 == 0 and int(element[0]) > 10:
            print('now at entry with id', element[0], 'tooks', (time.time() - start_time), 'seconds')
            estimated_similarities_np = np.array(estimated_similarities)
            real_similarities_np = np.array(real_similarities)
            pearson_score = pearsonr(estimated_similarities_np, real_similarities_np)
            print(pearson_score)
            np.save('real_similarites' + model_name, real_similarities_np)
            np.save('estimated_similarities' + model_name, estimated_similarities_np)
    print("finish")
    estimated_similarities_np = np.array(estimated_similarities)
    real_similarities_np = np.array(real_similarities)
    pearson_score = pearsonr(estimated_similarities_np, real_similarities_np)
    print("pearson_score", pearson_score)

    np.save('real_similarites' + model_name, real_similarities_np)
    np.save('estimated_similarities' + model_name, estimated_similarities_np)
    model_stat.append((model_name, pearson_score, none_counter))
    print('finished took', (time.time() - start_time), 'seconds for this model')
print(model_stat)
print('finished all models, took', (time.time() - start_time), 'seconds')



Test ldamodel_simple_mallet_20_10_keep_300000_gensimstop_topics200 :


AttributeError: 'LdaMallet' object has no attribute 'state'

In [8]:
import gensim
MODEL_BASE_DIR = 'F:/Not_Uploaded/conceptualization_eval/models/'
MODEL_FILE_EXTENSION = '.gensim'
lda.load(MODEL_BASE_DIR +'ldamodel_simple_mallet_20_10_keep_300000_gensimstop_topics200'+ MODEL_FILE_EXTENSION)

#gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda.ldamodel, gamma_threshold=0.001, iterations=50)

In [9]:
row_as_tuple = {'id': 'some_id', 'fb_user_id': 'fb_user', 'personwvch_id': 'personwvch', 'comparecounter': 12, 'datetime_of_matching': 'datetime'}

In [11]:
list_t = [row_as_tuple]

In [12]:
list_t

[{'comparecounter': 12,
  'datetime_of_matching': 'datetime',
  'fb_user_id': 'fb_user',
  'id': 'some_id',
  'personwvch_id': 'personwvch'}]

In [31]:
for idx, item in ((idx,item) for idx, item in enumerate(list_t) if item["personwvch_id"] == "personwvch"):
    print(idx, item)

0 {'datetime_of_matching': 'datetime', 'comparecounter': 12, 'fb_user_id': 'fb_user', 'personwvch_id': 'personwvch', 'id': 'some_id'}


0 {'datetime_of_matching': 'datetime', 'comparecounter': 12, 'fb_user_id': 'fb_user', 'personwvch_id': 'personwvch', 'id': 'some_id'}


In [32]:
list_t[0]

{'comparecounter': 12,
 'datetime_of_matching': 'datetime',
 'fb_user_id': 'fb_user',
 'id': 'some_id',
 'personwvch_id': 'personwvch'}